In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog 
from numpy.linalg import solve
import urllib.request
import os

### Downloading and parsing data

In [4]:
if not os.path.exists('data'):
    os.makedirs('data')

url = "http://www.nordpoolspot.com/globalassets/marketdata-excel-files/consumption-prognosis_2019_hourly.xls" 
urllib.request.urlretrieve(url, 'data/consumption.xls');

url2 = "http://www.nordpoolspot.com/globalassets/marketdata-excel-files/wind-power-dk-prognosis_2019_hourly.xls" 
urllib.request.urlretrieve(url2, 'data/wind-prognosis.xls');

consumptionProg_raw_data = pd.read_html('data/consumption.xls')[0].values
consumptionProg_df = pd.DataFrame(consumptionProg_raw_data)
consumptionProg_df.columns = ['Dates', 'Hours', 'NO', 'SE', 'FI', 'DK1', 'DK2', 'EE', 'LV', 'LT']
consumptionProg_df = consumptionProg_df[['Dates', 'Hours', 'DK1', 'DK2']]
consumptionProg_df['Hours'] = consumptionProg_df['Hours'].str.slice(stop=2)
consumptionProg_df['ts'] = pd.to_datetime(consumptionProg_df['Dates'] + consumptionProg_df['Hours'] + ':00', format='%d-%m-%Y%H:%M')
consumptionProg_df = consumptionProg_df[consumptionProg_df['ts'] < pd.to_datetime('2019-02-01')]

## Including Import of DK1 -> Norway 200MW
consumptionProg_df['DK1'] = consumptionProg_df['DK1'] + 200

## Including Export of DK1 -> Germany 100MW
for i in consumptionProg_df.index:
    if (consumptionProg_df['ts'][i].time() >= pd.to_datetime('07:00:00').time()) & (consumptionProg_df['ts'][i].time() < pd.to_datetime('14:00:00').time()):
        consumptionProg_df['DK1'][i] = consumptionProg_df['DK1'][i] - 100
        
## Including Import of DK2 -> Sweden 150MW
for i in consumptionProg_df.index:
    if (consumptionProg_df['ts'][i].time() >= pd.to_datetime('13:00:00').time()) & (consumptionProg_df['ts'][i].time() < pd.to_datetime('19:00:00').time()):
        consumptionProg_df['DK2'][i] = consumptionProg_df['DK2'][i] + 150

windProg_raw_data = pd.read_html('data/wind-prognosis.xls')[0].values
windProg_df = pd.DataFrame(windProg_raw_data)
windProg_df.columns = ['Dates', 'Hours', 'DK1', 'DK2']
windProg_df['Hours'] = windProg_df['Hours'].str.slice(stop=2)
windProg_df['ts'] = pd.to_datetime(windProg_df['Dates'] + windProg_df['Hours'] + ':00', format='%d-%m-%Y%H:%M')
windProg_df = windProg_df[windProg_df['ts'] < pd.to_datetime('2019-02-01')]

windProg_df['WW1'] = windProg_df['DK1']*(1/4)
windProg_df['WW2'] = windProg_df['DK1']*(3/4)
windProg_df['EW1'] = windProg_df['DK2']*(1/3)
windProg_df['EW2'] = windProg_df['DK2']*(2/3)

supply_all = np.array([[400, 330, 345, 390, 510, 1000, 900, 1200, 320, 360, 400, 350, 730, 630, 800],
                       [70, 64, 153, 82, 89, 25, 250, 19, 43, 39, 36, 31, 5, 10, 250]])

supply_all_df = pd.DataFrame(supply_all.T).rename(columns={0:'ID', 1:'Supply', 2:'Cost'})

supply_restricted= np.array([[400, 330, 345, 390, 510, 900, 320, 360, 400, 350, 730, 630, 800],
                             [70, 64, 153, 82, 89, 250, 43, 39, 36, 31, 5, 10, 250]])

supply_restricted_df = pd.DataFrame(supply_restricted.T).rename(columns={0:'ID', 1:'Supply', 2:'Cost'})


d:\lucas\123\ren-elec-markets\assignment1\venv\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\lucas\123\ren-elec-markets\assignment1\venv\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
for i in windProg_df.index:
    
    power_d = consumptionProg_df.values[i,2:4]

    if (windProg_df['ts'][i].time() >= pd.to_datetime('05:00:00').time()) & (windProg_df['ts'][i].time() < pd.to_datetime('22:00:00').time()):
        power_g = np.concatenate((supply_restricted[0], windProg_df.values[i,5:]))
        cost_g = np.concatenate((supply_restricted[1], [0 , 0, 0, 0]))
    else:
        power_g = np.concatenate((supply_all[0], windProg_df.values[i,5:]))
        cost_g = np.concatenate((supply_restricted[1], [0 , 0, 0, 0]))

    print(windProg_df['ts'][i], '- Units: ', power_g.shape[0])
    print('Pg: ', power_g)
    print('Cg: ', cost_g)
    print('Pd: ', power_d)

2019-01-01 00:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 877.5
 2632.5 326.0 652.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2057 1359]
2019-01-01 01:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 883.75
 2651.25 339.0 678.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2019 1321]
2019-01-01 02:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 855.25
 2565.75 338.3333333333333 676.6666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [1955 1296]
2019-01-01 03:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 803.25
 2409.75 330.3333333333333 660.6666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [1909 1242]
2019-01-01 04:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 90

Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 213.25 639.75 149.0
 298.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2652 1620]
2019-01-03 07:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 221.0 663.0
 159.33333333333331 318.66666666666663]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2964 1886]
2019-01-03 08:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 215.25 645.75
 153.66666666666666 307.3333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3136 1997]
2019-01-03 09:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 211.0 633.0
 143.33333333333331 286.66666666666663]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3099 2014]
2019-01-03 10:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 180.75 542.25 127.0
 254.

2019-01-05 02:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 623.75
 1871.25 181.66666666666666 363.3333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [1949 1210]
2019-01-05 03:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 557.5
 1672.5 171.66666666666666 343.3333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [1973 1202]
2019-01-05 04:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 536.25
 1608.75 156.66666666666666 313.3333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2030 1216]
2019-01-05 05:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 526.25 1578.75
 151.33333333333331 302.66666666666663]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2070 1242]
2019-01-05 06:00:00 - U

Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 173.0 519.0
 29.333333333333332 58.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2068 1305]
2019-01-07 05:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 181.25 543.75
 24.333333333333332 48.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2214 1367]
2019-01-07 06:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 188.25 564.75
 24.333333333333332 48.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2588 1579]
2019-01-07 07:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 186.0 558.0
 30.666666666666664 61.33333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2974 1860]
2019-01-07 08:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 7

2019-01-08 17:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 915.25 2745.75
 348.3333333333333 696.6666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3382 2383]
2019-01-08 18:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 901.0 2703.0 355.0
 710.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3229 2329]
2019-01-08 19:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 927.75 2783.25
 367.3333333333333 734.6666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3081 2044]
2019-01-08 20:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 910.0 2730.0
 362.3333333333333 724.6666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2900 1905]
2019-01-08 21:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 6

...

2019-01-20 11:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 53.0 159.0
 13.333333333333332 26.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2516 1869]
2019-01-20 12:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 49.25 147.75
 16.333333333333332 32.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2451 1830]
2019-01-20 13:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 47.0 141.0
 18.333333333333332 36.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2417 1949]
2019-01-20 14:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 48.0 144.0 20.0 40.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2508 1930]
2019-01-20 15:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 

2019-01-23 15:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 204.25 612.75
 92.33333333333333 184.66666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3163 2155]
2019-01-23 16:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 199.5 598.5 99.0
 198.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3216 2253]
2019-01-23 17:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 211.0 633.0
 116.33333333333333 232.66666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3439 2425]
2019-01-23 18:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 220.0 660.0 142.0
 284.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3344 2359]
2019-01-23 19:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 223.0 669.0 138.0
 27

Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2188 1393]
2019-01-26 05:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 185.5 556.5
 17.666666666666664 35.33333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2234 1423]
2019-01-26 06:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 193.25 579.75
 13.333333333333332 26.666666666666664]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2328 1493]
2019-01-26 07:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 189.5 568.5
 11.666666666666666 23.333333333333332]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2434 1610]
2019-01-26 08:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 183.0 549.0 11.0 22.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2590 17

2019-01-28 00:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 158.75
 476.25 46.666666666666664 93.33333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2159 1344]
2019-01-28 01:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 148.5 445.5
 45.666666666666664 91.33333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2121 1283]
2019-01-28 02:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 135.5 406.5
 47.666666666666664 95.33333333333333]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2102 1264]
2019-01-28 03:00:00 - Units:  19
Pg:  [400 330 345 390 510 1000 900 1200 320 360 400 350 730 630 800 130.0 390.0
 62.33333333333333 124.66666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [2134 1292]
2019-01-28 04:00:00 - 

Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 484.0 1452.0 235.0
 470.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3076 2136]
2019-01-30 16:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 528.0 1584.0 254.0
 508.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3108 2221]
2019-01-30 17:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 572.0 1716.0 275.0
 550.0]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3399 2420]
2019-01-30 18:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 610.25 1830.75
 295.3333333333333 590.6666666666666]
Cg:  [ 70  64 153  82  89 250  43  39  36  31   5  10 250   0   0   0   0]
Pd:  [3304 2381]
2019-01-30 19:00:00 - Units:  17
Pg:  [400 330 345 390 510 900 320 360 400 350 730 630 800 628.5 1885.5
 294.3333333333333 588.6666666666666]
Cg:  [ 70  64 153  82  89

In [7]:
windProg_df

,Dates,Hours,DK1,DK2,ts,WW1,WW2,EW1,EW2
0,01-01-2019,00,3510,978,2019-01-01 00:00:00,877.5,2632.5,326,652
1,01-01-2019,01,3535,1017,2019-01-01 01:00:00,883.75,2651.25,339,678
2,01-01-2019,02,3421,1015,2019-01-01 02:00:00,855.25,2565.75,338.333,676.667
3,01-01-2019,03,3213,991,2019-01-01 03:00:00,803.25,2409.75,330.333,660.667
4,01-01-2019,04,3222,984,2019-01-01 04:00:00,805.5,2416.5,328,656
5,01-01-2019,05,3306,1032,2019-01-01 05:00:00,826.5,2479.5,344,688
6,01-01-2019,06,3290,1057,2019-01-01 06:00:00,822.5,2467.5,352.333,704.667
7,01-01-2019,07,3157,1049,2019-01-01 07:00:00,789.25,2367.75,349.667,699.333
8,01-01-2019,08,2953,1012,2019-01-01 08:00:00,738.25,2214.75,337.333,674.667
9,01-01-2019,09,2848,990,2019-01-01 09:00:00,712,2136,330,660


### Standard Model

In [ ]:
A_eq = np.array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 1.,  1., 
                  -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]])
b_eq = 0

A = np.identity(A_eq.size)

power_g = np.array([120, 50, 200, 400, 60, 50, 60, 100, 70, 50, 70, 45, 50, 60, 50]) 
power_d = np.array([250, 300, 120, 80, 40, 70, 60, 45, 30, 35, 25, 10]) 
b = np.array([np.concatenate((power_g, power_d))]).T

cost_g = np.array([0, 0, 15, 30, 32.5, 34, 36, 37.5, 39, 40, 60, 70, 100, 150, 200])
cost_d = -np.array([200, 110, 100, 90, 85, 75, 65, 40, 38, 31, 24, 16])
c = np.concatenate((cost_g, cost_d))

res_std = linprog(c, A_eq=A_eq, b_eq = b_eq, A_ub=A, b_ub=b, bounds=(0, None))
print('Generation schedules:', res_std.x[:15])
print('Demand schedules:', res_std.x[15:])

### Dual

In [ ]:
a1 = np.ones(15)
a2 = -np.ones(12)
a3 = np.array([np.hstack((a1,a2))]).T
a4 = -np.identity(27)

A_tilde = np.concatenate((a4,a3),axis=1)

b_tilde = np.concatenate((cost_g, cost_d))

c_tilde = np.append(-b, np.array([0]))

res_dual = linprog(-c_tilde, A_ub=A_tilde, b_ub=b_tilde)
print('Cost:', res_dual.x[-1])

### Suppliers Dictionary

In [ ]:
suppliers_dict = {'G1' : 'FlexiGas',
                 'G2' : 'FlexiGas',
                 'G3' : 'FlexiGas',
                 'G4' : 'Peako',
                 'G5' : 'Peako',
                 'G6' : 'Nuke22',
                 'G7' : 'CoalAtLast',
                 'G8' : 'Nuke22',
                 'G9' : 'RoskildeCHP',
                 'G10' : 'RoskildeCHP',
                 'G11' : 'Avedovre',
                 'G12' : 'Avedovre',
                 'G13' : 'BlueWater',
                 'G14' : 'BlueWater',
                 'G15' : 'CoalAtLast'}